## Golomb编码

## 游程编码

## 总程序

初始化变量和变量组

In [193]:
# for 8 bits picture
MAXVAL=255
RANGE=256
bpp=8
qbpp=8
LIMIT=32
MAX_C=127
MIN_C=-128

# A，N从0到366，B，C从0到364，Nn从365到366，A初始化为全4，其余初始化都为全0
A = [4 for _ in range(366)]
B = [0 for _ in range(364)]
C = [0 for _ in range(364)]
N = [0 for _ in range(366)]
Nn= {365:0,366:0}

RUNcnt = 0
RUNindex = 0

J = [0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 9, 10, 11, 12, 13, 14, 15]

# pic(index starts from 1)
# 0     0       90      74
# 68    50      43      205
# 64    145     145     145
# 100   145     145     145

origin_pic = [[0,0,90,74],
[68,50,43,205],
[64,145,145,145],
[100,145,145,145]]

自定义函数

In [194]:
def reconstruction_pic(origin_pic):
    expand_pic = [[0 for _ in range(len(origin_pic[0])+2)] for _ in range(len(origin_pic)+1)]
    for i in range(len(origin_pic)):
        for j in range(len(origin_pic[0])):
            expand_pic[i+1][j+1]=origin_pic[i][j]
    
    for i in range(len(origin_pic)-1):
        expand_pic[i+2][0]=origin_pic[i][0]
        expand_pic[i+1][len(origin_pic[0])+1]=origin_pic[i][len(origin_pic[0])-1]
    return expand_pic

def get_pixels(expand_pic,hang_index,lie_index):
    _hang_index = hang_index+1
    _lie_index = lie_index+1

    x = expand_pic[_hang_index     ][_lie_index      ]
    a = expand_pic[_hang_index     ][_lie_index    - 1  ]
    b = expand_pic[_hang_index  -1 ][_lie_index    ]
    c = expand_pic[_hang_index - 1 ][_lie_index   -1 ]
    d = expand_pic[_hang_index -1  ][_lie_index  +1  ]
    return x,a,b,c,d

def get_gradient(a,b,c,d):
    D1 = d-b
    D2 = b-c
    D3 = c-a
    return D1,D2,D3

def mode_choose(D1,D2,D3):
    '''
    return mode = 0 : run mode
    return mode = 1 : regular mode
    '''
    if D1==0 and D2==0 and D3==0:
        mode = 0
    else:
        mode = 1
    return mode



regular mode functions

In [195]:
def quantinize(D):
    if D<=-21:
        Q=-4
    elif D<=-7:
        Q=-3
    elif D<=-3:
        Q=-2
    elif D<0:
        Q=-1
    elif D==0:
        Q=0
    elif D<=3:
        Q=1
    elif D<=7:
        Q=2
    elif D<=21:
        Q=3
    else:
        Q=4
    return Q

def mapping_and_sign(Q1,Q2,Q3):
    if Q1<0 or (Q1==0 and Q2<0) or (Q1==0 and Q2==0 and Q3<0):
        SIGN = -1
        Q1,Q2,Q3=-Q1,-Q2,-Q3
    else:
        SIGN = 1
    Q = 81*Q1+9*Q2+Q3
    return Q,SIGN

def prediction(a,b,c):
    if c>max(a,b):
        Px = min(a,b)
    elif c<min(a,b):
        Px = max(a,b)
    else:
        Px = a+b-c
    return Px

def prediction_corrected(Px,SIGN,Q,MAXVAL=255):
    Px = Px + SIGN*C[Q]
    if(Px>MAXVAL):
        Px = MAXVAL
    if(Px<0):
        Px = 0
    return Px

def prediction_error(x,Px,SIGN):
    Errval = (x-Px)*SIGN
    return Errval

def prediction_error_modulo_reduction(Errval,RANGE=256):
    if Errval<(-RANGE/2):
        Errval+=RANGE
    elif Errval>((RANGE+1)/2):
        Errval-=RANGE
    return Errval

def Golomb_k(Q):
    global A,B,C,N
    k=0
    while((N[Q]<<k)<A[Q] and k<=7):
        k+=1
    return k

def error_mapping(Errval,Q,k):
    if(k==0 and 2*B[Q]<=-N[Q]):# special mapping
        if(Errval >= 0):
            MErrval = 2*Errval+1
        else:
            MErrval = -2*(Errval+1)
    else:# regular mapping
        if Errval>=0:
            MErrval = 2*Errval
        else:
            MErrval = -2*Errval+1
    return MErrval

def update_params(Q,Errval):
    global A,B,C,N,RESET,MAX_C,MIN_C
    B[Q] = B[Q] + Errval
    A[Q] = (A[Q] + Errval) if (Errval>=0) else (A[Q] - Errval)
    if N[Q]==RESET:
        A[Q] = A[Q] >> 1
        if B[Q] >= 0:
            B[Q] = B[Q] >> 1
        else:
            B[Q] = -((1-B[Q])>>1)
        N[Q]>>1
    N[Q]=N[Q]+1

    if B[Q]<=-N[Q]:
        B[Q]=B[Q]+N[Q]
        if C[Q]>MIN_C:
            C[Q] = C[Q] - 1
        if B[Q]<=-N[Q]:
            B[Q] = -N[Q] + 1
    elif B[Q]>0:
        B[Q] = B[Q]-N[Q]
        if C[Q] < MAX_C:
            C[Q] = C[Q]+1
        if B[Q]>0:
            B[Q]=0
            

def bin8(num):
    outnum = bin(num)[2:]
    while(len(outnum)!=8):
        outnum = '0' + outnum
    return outnum

def one_element_coding(num):
    outnum = '0' * num + '1'
    return outnum

def golomb_coding(MErrval,k,LIMIT=32,qbpp=8):
    MErrval_R_k = MErrval>>k
    outstr = ''
    if MErrval_R_k < LIMIT-qbpp-1:
        outstr+=one_element_coding(MErrval_R_k)
        outstr+=bin8(MErrval)[-k:]
    else:
        outstr+=one_element_coding(LIMIT-qbpp-1)
        outstr+=bin8(MErrval-1)[-qbpp:]
    return outstr

def golomb_decoding(outstr,k,LIMIT=32,qbpp=8):
    pass

In [196]:
def regular_mode(x,a,b,c,d):
    D1,D2,D3 = get_gradient(a,b,c,d)

    Q,SIGN = mapping_and_sign(quantinize(D1),quantinize(D2),quantinize(D3))

    Px = prediction(a,b,c)

    Px = prediction_corrected(Px,SIGN,Q)

    Errval = prediction_error(x,Px,SIGN)

    Errval = prediction_error_modulo_reduction(Errval)

    k = Golomb_k(Q)

    MErrval = error_mapping(Errval,Q,k)

    regular_outstream = golomb_coding(MErrval,k)

    update_params(Q,Errval)
    return regular_outstream


In [222]:
Pixel_cnt=0
def GetNextSample():
    global Pixel_cnt,expand_pic,origin_pic
    i = Pixel_cnt//(len(origin_pic[0]))
    j = Pixel_cnt%(len(origin_pic[0]))
    if j==len(origin_pic[0])-1:
        EOL = 1
    x,a,b,c,d = get_pixels(expand_pic, i,j)
    Pixel_cnt +=1
    return x,a,b,c,d,EOL

In [197]:
if __name__ == "__main__":

    # for 8 bits picture
    MAXVAL=255
    RANGE=256
    bpp=8
    qbpp=8
    LIMIT=32
    MAX_C=127
    MIN_C=-128
    RESET = 64
    MAX_C = 127
    MIN_C = -128

    # A，N从0到366，B，C从0到364，Nn从365到366，A初始化为全4，其余初始化都为全0
    A = [4 for _ in range(366)]
    B = [0 for _ in range(364)]
    C = [0 for _ in range(364)]
    N = [0 for _ in range(366)]
    Nn= {365:0,366:0}

    RUNcnt = 0
    RUNindex = 0
    RUNval = 0
    J = [0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 9, 10, 11, 12, 13, 14, 15]

    EOL=0
    # pic(index starts from 1)
    # 0     0       90      74
    # 68    50      43      205
    # 64    145     145     145
    # 100   145     145     145

    # origin_pic = [[0,0,90,74],
    # [68,50,43,205],
    # [64,145,145,145],
    # [100,145,145,145]]

    origin_pic = [[1,2,3,4],
    [5,6,43,205],
    [64,145,9,10],
    [11,12,13,145]]

    expand_pic = reconstruction_pic(origin_pic)

    outstream = ''
    

    for i in range(len(origin_pic)):
        for j in range(len(origin_pic[0])):
            x,a,b,c,d = get_pixels(expand_pic, i,j)
            D1,D2,D3 = get_gradient(a,b,c,d)
            mode = mode_choose(D1,D2,D3)
            
            if mode==1:#regular mode
                outstream += regular_mode(x,a,b,c,d)
            elif mode==0 and x==a:#run mode
                RUNcnt += 1
            # elif mode==0 and j==len(origin_pic[0])-1:
            #     outstream += run_mode_EOL()# end of line
            # elif mode==0:
            #     outstream += run_mode_BP()# break point
print(outstream)

1000000111000100000011100001000100000010000000000100011011110110111011011010001011111000011011111011101101110100000100001000111111001


In [208]:
def bitstream(num,width):
    num_2 = bin(num)[2:]
    if len(num_2)>width:
        raise Exception("width is not enough for num!")
    while(len(num_2)!=width):
        num_2='0'+num_2
    return num_2

def run_mode_EOL():
    global RUNcnt,RUNindex
    run_outstream_EOL = ''
    while(RUNcnt >=(1<<J[RUNindex])):
        run_outstream_EOL += '1'
        RUNcnt -= 1<<J[RUNindex]
        if(RUNindex<31):
            RUNindex+=1
    if(RUNcnt>0):
        run_outstream_EOL += '1'
    return run_outstream_EOL

def run_mode_BP():
    run_outstream_BP = '0'
    run_outstream_BP += 


raise Exception("value is too large")